In [ ]:
import pandas as pd
import json
import polars as pl
from polars import col, lit
import altair as alt

In [ ]:
with open("../../output/last_run_tabular.json") as f:
    data = json.load(f)
agents = pl.DataFrame(data["agents"])
markets = pl.DataFrame(data["markets"]).select(pl.exclude("pricer"))
actions = pl.DataFrame(data["actions"])

In [ ]:
actions.head(3)

In [ ]:
markets.head(3)

In [ ]:
markets.groupby(col("tick")).agg((pl.median("price").alias("median"), pl.sum("price").alias("sum")))

In [ ]:
alt.Chart(markets).mark_point().encode(x="tick:Q", y="price:Q")


In [ ]:
alt.Chart(agents).mark_point().encode(x="tick:Q", y="coins:Q")

In [ ]:
agents.join(actions, left_on="id", right_on="agent_id").select(pl.exclude("id")).head(3)